In [2]:
import os
from os.path import dirname

root_path = dirname(dirname(os.getcwd()))
print(root_path)
import sys

sys.path.append(root_path + "/DuongNA/2_Scripts/")
import pandas as pd
import numpy as np
import copy
import time
import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render

from Event_log_processing_utils import (
    Extract_trace_and_temporal_features,
    Extract_prefix,
)
from sklearn.preprocessing import OneHotEncoder
import warnings

warnings.filterwarnings("ignore")

data_dir = root_path + "/DuongNA/1_Data/"
project_dir = root_path + "/DuongNA/"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

/home/sebdis/ProcessMining/Next_Activity_GNN
/home/sebdis/ProcessMining/Next_Activity_GNN
cuda:0


In [3]:
import random

torch.manual_seed(0)
torch.cuda.manual_seed(0)
random.seed(0)
np.random.seed(0)

## 1. Load data

In [4]:
data_name = "Helpdesk"
# data_name = "env_permit"
# data_name = "BPI_Challenge_2012_A"
# data_name = "BPI_Challenge_2012_O"
# data_name = "BPI_Challenge_2012_W_Complete"
# data_name = "BPI_Challenge_2013_closed_problems"
# data_name = "BPI_Challenge_2013_incidents"

In [5]:
tab_all = pd.read_csv(data_dir + data_name + "_processed_all.csv")
tab_train = pd.read_csv(data_dir + data_name + "_processed_train.csv")
tab_valid = pd.read_csv(data_dir + data_name + "_processed_valid.csv")
tab_test = pd.read_csv(data_dir + data_name + "_processed_test.csv")

In [6]:
# Statistics of the dataset
lines, lines_t, lines_t2, lines_t3, lines_t4 = Extract_trace_and_temporal_features(
    tab_all
)
print("num_cases: {}".format(len(tab_all["Case_ID"].unique())))
print("num_activities: {}".format(len(tab_all["Activity"].unique())))
print("num_events: {}".format(len(tab_all)))
avglen = round(np.mean([len(x) for x in lines]), 2)
print("avg_case_len: {}".format(avglen))
maxlen = max([len(x) for x in lines])  # find maximum line size
print("max_case_len: {}".format(maxlen))
print(
    "avg_case_duration: {}".format(
        round(np.mean([sublist[-1] for sublist in lines_t2]) / 86400, 2)
    )
)
print(
    "max_case_duration: {}".format(
        round(max([sublist[-1] for sublist in lines_t2]) / 86400, 2)
    )
)
print(
    "min_case_duration: {}".format(
        round(min([sublist[-1] for sublist in lines_t2]) / 86400, 2)
    )
)
list_unique_line = []
for line in lines:
    if line not in list_unique_line:
        list_unique_line.append(line)
print("variants: {}".format(len(list_unique_line)))

num_cases: 4552
num_activities: 10
num_events: 21197
avg_case_len: 4.66
max_case_len: 15
avg_case_duration: 40.85
max_case_duration: 59.99
min_case_duration: 30.63
variants: 207


## 2. Prepare inputs and outputs for model training

In [7]:
def Prepare_X_Y_remaining_time(
    tab, list_activities, divisor, divisor2, divisor_rt, encoder, maxlen
):
    lines, lines_t, lines_t2, lines_t3, lines_t4 = Extract_trace_and_temporal_features(
        tab
    )
    prefixes, outputs = Extract_prefix(lines, lines_t, lines_t2, lines_t3, lines_t4)
    num_samples = len(prefixes[0])
    #     [sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4], [next_ope, next_ope_t, end_ope_t]
    print("Vectorization...")
    num_features = len(list_activities) + 5  # 1 order feature + 4 temporal features
    print("num features: {}".format(num_features))
    X = np.zeros((num_samples, maxlen, num_features), dtype=np.float32)
    Y = np.zeros((num_samples, len(list_activities)), dtype=np.float32)
    for i, sentence in enumerate(prefixes[0]):

        leftpad = maxlen - len(sentence)
        end_t = outputs[2][i]
        sentence_t = prefixes[1][i]
        sentence_t2 = prefixes[2][i]
        sentence_t3 = prefixes[3][i]
        sentence_t4 = prefixes[4][i]
        one_hot_act_matrix = encoder.transform(
            np.array(sentence).reshape((len(sentence), 1))
        ).toarray()
        for t, char in enumerate(sentence):
            X[i, t + leftpad, : len(list_activities)] = one_hot_act_matrix[t, :]
            X[i, t + leftpad, len(list_activities)] = (
                t + 1
            )  # order of the activity in the sequence {1,...,maxlen}
            X[i, t + leftpad, len(list_activities) + 1] = sentence_t[t] / divisor
            X[i, t + leftpad, len(list_activities) + 2] = sentence_t2[t] / divisor2
            X[i, t + leftpad, len(list_activities) + 3] = sentence_t3[t] / 86400
            X[i, t + leftpad, len(list_activities) + 4] = sentence_t4[t] / 7
        # print(encoder.transform(np.array([[outputs[0][i]]])).toarray())
        Y[i] = encoder.transform(np.array([[outputs[0][i]]])).toarray()[0]
        # break

    return X, Y

In [8]:
list_activities = list(tab_all["Activity"].unique())
num_features = len(list_activities) + 5
# creating instance of one-hot-encoder and fit on the whole dataset
encoder = OneHotEncoder(handle_unknown="ignore")
encoder.fit(np.array(list_activities).reshape((len(list_activities), 1)))

lines, lines_t, lines_t2, lines_t3, lines_t4 = Extract_trace_and_temporal_features(
    tab_all
)
maxlen = max([len(x) for x in lines])  # find maximum line size
lines, lines_t, lines_t2, lines_t3, lines_t4 = Extract_trace_and_temporal_features(
    tab_train
)
divisor = np.mean(
    [item for sublist in lines_t for item in sublist]
)  # average time between events
print("divisor: {}".format(divisor))
divisor2 = np.mean(
    [item for sublist in lines_t2 for item in sublist]
)  # average time between current and first events
print("divisor2: {}".format(divisor2))
prefixes, outputs = Extract_prefix(lines, lines_t, lines_t2, lines_t3, lines_t4)
divisor_rt = np.mean(outputs[2])
print("divisor_rt: {}".format(divisor_rt))
# Train data
X_train, Y_train = Prepare_X_Y_remaining_time(
    tab_train, list_activities, divisor, divisor2, divisor_rt, encoder, maxlen
)
# Valid data
X_valid, Y_valid = Prepare_X_Y_remaining_time(
    tab_valid, list_activities, divisor, divisor2, divisor_rt, encoder, maxlen
)
# Test data
X_test, Y_test = Prepare_X_Y_remaining_time(
    tab_test, list_activities, divisor, divisor2, divisor_rt, encoder, maxlen
)

divisor: 744287.3818275507
divisor2: 1111494.5871693576
divisor_rt: 2966515.512665685
Vectorization...
num features: 15
Vectorization...
num features: 15
Vectorization...
num features: 15


In [9]:
class EventLogData(Dataset):
    def __init__(self, input_x, output):
        self.X = input_x
        self.y = output
        self.y = self.y.to(torch.float32)

    # get the number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at a particular index in the dataset
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

In [10]:
valid_loader = DataLoader(
    EventLogData(torch.tensor(X_valid), torch.tensor(Y_valid)),
    batch_size=X_valid.shape[0],
    shuffle=False,
)
test_loader = DataLoader(
    EventLogData(torch.tensor(X_test), torch.tensor(Y_test)),
    batch_size=1,
    shuffle=False,
)

In [11]:
# A Class to keep track of the metrics of the classification process
class ClassificationMetrics:

    # Constructor takes the number of classes, in our case 20
    def __init__(self, num_classes=20):
        self.num_classes = num_classes
        # Initialize a confusion matrix
        self.C = torch.zeros(num_classes, num_classes)

    # Update the confusion matrix with the new scores
    def add(self, yp, yt):
        # yp: 1D tensor with predictions
        # yt: 1D tensor with ground-truth targets
        yp = yp.to("cpu")
        yt = yt.to("cpu")
        with torch.no_grad():  # We require no computation graph
            self.C += (
                (yt * self.C.shape[1] + yp)
                .bincount(minlength=self.C.numel())
                .view(self.C.shape)
                .float()
            )

    def clear(self):
        # We set the confusion matrix to zero
        self.C.zero_()

    # Computes the global accuracy
    def acc(self):
        return self.C.diag().sum().item() / self.C.sum()

    # Computes the class-averaged accuracy
    def mAcc(self):
        return (self.C.diag() / self.C.sum(-1)).mean().item()

    # Computers the class-averaged Intersection over Union
    def mIoU(self):
        return (
            (self.C.diag() / (self.C.sum(0) + self.C.sum(1) - self.C.diag()))
            .mean()
            .item()
        )

    # Returns the confusion matrix
    def confusion_matrix(self):
        return self.C

## 3. Hyperparameter tuning with Ax package

In [16]:
# Creating the LSTM class
class LSTM_direct(nn.Module):
    #  Determine what layers and their order in CNN object
    def __init__(self, parameterization):
        super(LSTM_direct, self).__init__()
        self.hidden_dim = parameterization.get("neurons", 40)
        self.num_layers = parameterization.get("layers", 1)
        self.droppout_prob = parameterization.get("dropout", 0.2)

        self.lstm = nn.LSTM(
            input_size=num_features,
            hidden_size=self.hidden_dim,
            num_layers=self.num_layers,
            batch_first=True,
            dropout=self.droppout_prob,
        )
        self.fc = nn.Linear(self.hidden_dim, len(list_activities))

    # Progresses data across layers
    def forward(self, x):
        batch_size = x.size(0)
        init_states, init_cells = self.init_hidden(batch_size)
        init_states = init_states.to(x.device)
        init_cells = init_cells.to(x.device)
        _, (last_Hidden_State, _) = self.lstm(x, (init_states, init_cells))
        out = self.fc(last_Hidden_State[-1])
        return out

    def init_hidden(self, batch_size):
        init_states = []
        init_cells = []
        for _ in range(self.num_layers):
            init_states.append(torch.zeros(batch_size, self.hidden_dim))
            init_cells.append(torch.zeros(batch_size, self.hidden_dim))
        return torch.stack(init_states, dim=0), torch.stack(
            init_cells, dim=0
        )  # (num_layers, B, H)


def net_train(
    net, train_loader, valid_loader, parameters, dtype, device, early_stop_patience
):
    net.to(dtype=dtype, device=device)
    min_delta = 0
    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        net.parameters(), lr=parameters.get("lr", 0.001)
    )  # 0.001 is used if no lr is specified
    num_epochs = 100  # Play around with epoch number
    metric_tracker = ClassificationMetrics(len(list_activities))
    # Train Network
    not_improved_count = 0
    start_time = time.time()
    for epoch in range(num_epochs):
        net.train()
        training_loss = 0
        num_train = 0

        metric_tracker.clear()
        for inputs, labels in train_loader:
            # move data to proper dtype and device
            inputs = inputs.to(dtype=dtype, device=device)
            labels = torch.tensor([torch.max(yi, 0)[1] for yi in labels])
            labels = labels.to(device=device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            output = net(inputs)
            loss = criterion(output, labels)
            # back prop
            loss.backward()
            # optimize
            optimizer.step()
            training_loss += loss.item()
            num_train += 1
            _, preds = torch.max(output, 1)
            preds = preds.to(device)
            metric_tracker.add(preds, labels)
        train_acc = metric_tracker.acc()
        metric_tracker.clear()

        with torch.no_grad():
            net.eval()
            num_valid = 0
            validation_loss = 0
            for i, (inputs, targets) in enumerate(valid_loader):
                inputs, targets = inputs.to(device), torch.tensor(
                    [torch.max(yi, 0)[1] for yi in targets]
                ).to(device)
                yhat_valid = net(inputs)
                loss_valid = criterion(yhat_valid, targets)
                validation_loss += loss_valid.item()
                num_valid += 1
                _, preds = torch.max(yhat_valid, 1)
                preds = preds.to(device)
                metric_tracker.add(preds, targets)
        val_acc = metric_tracker.acc()

        avg_training_loss = training_loss / num_train
        avg_validation_loss = validation_loss / num_valid

        print(
            "Epoch: {}, Training Loss | Acc: {:.3f} {:.3f}, Validation Loss | ACC : {:.3f} {:.3f}".format(
                epoch, avg_training_loss, train_acc, avg_validation_loss, val_acc
            )
        )
        if epoch == 0:
            best_loss = avg_validation_loss
            best_model = copy.deepcopy(net)
        else:
            if best_loss - avg_validation_loss >= min_delta:
                best_model = copy.deepcopy(net)
                best_loss = avg_validation_loss
                not_improved_count = 0
            else:
                not_improved_count += 1
        # Early stopping
        if not_improved_count == early_stop_patience:
            print(
                "Validation performance didn't improve for {} epochs. "
                "Training stops.".format(early_stop_patience)
            )
            break
    training_time = time.time() - start_time
    print("Training time:", training_time)
    return best_model


def lstm_direct_evaluate(net, data_loader, dtype, device):
    criterion = nn.CrossEntropyLoss()
    net.eval()
    loss = 0
    total = 0
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(data_loader):
            # move data to proper dtype and device
            inputs = inputs.to(dtype=dtype, device=device)
            targets = torch.tensor([torch.max(yi, 0)[1] for yi in targets]).to(
                device=device
            )
            outputs = net(inputs)
            loss += criterion(outputs, targets)
            total += 1
    return loss.item() / total


def train_evaluate(parameterization):

    # constructing a new training data loader allows us to tune the batch size
    train_loader = DataLoader(
        EventLogData(torch.tensor(X_train), torch.tensor(Y_train)),
        batch_size=parameterization.get("batchsize", 32),
        shuffle=True,
    )

    # Get neural net
    untrained_net = LSTM_direct(parameterization)
    # train
    trained_net = net_train(
        net=untrained_net,
        train_loader=train_loader,
        valid_loader=valid_loader,
        parameters=parameterization,
        dtype=dtype,
        device=device,
        early_stop_patience=10,
    )

    # return the accuracy of the model as it was trained in this run
    return lstm_direct_evaluate(
        net=trained_net,
        data_loader=valid_loader,
        dtype=dtype,
        device=device,
    )

In [17]:
dtype = torch.float
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
best_parameters, values, experiment, model = optimize(
    parameters=[
        {
            "name": "neurons",
            "type": "choice",
            "values": [40, 60, 80, 100],
            "value_type": "int",
        },
        {
            "name": "layers",
            "type": "choice",
            "values": [2, 3, 4, 5],
            "value_type": "int",
        },
        {
            "name": "lr",
            "type": "range",
            "bounds": [1e-4, 0.1],
            "value_type": "float",
            "log_scale": True,
        },
        {
            "name": "batchsize",
            "type": "choice",
            "values": [16, 32, 64],
            "value_type": "int",
        },
        {"name": "dropout", "type": "range", "bounds": [0, 0.5], "value_type": "float"},
    ],
    evaluation_function=train_evaluate,
    objective_name="CrossEntropy loss",
    minimize=True,
    random_seed=23,
    total_trials=1,
)

print(best_parameters)
means, covariances = values
print(means)

[INFO 11-07 11:14:34] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[ChoiceParameter(name='neurons', parameter_type=INT, values=[40, 60, 80, 100], is_ordered=True, sort_values=True), ChoiceParameter(name='layers', parameter_type=INT, values=[2, 3, 4, 5], is_ordered=True, sort_values=True), RangeParameter(name='lr', parameter_type=FLOAT, range=[0.0001, 0.1], log_scale=True), ChoiceParameter(name='batchsize', parameter_type=INT, values=[16, 32, 64], is_ordered=True, sort_values=True), RangeParameter(name='dropout', parameter_type=FLOAT, range=[0.0, 0.5])], parameter_constraints=[]).
[INFO 11-07 11:14:34] ax.modelbridge.dispatch_utils: Using Models.BOTORCH_MODULAR since there are more ordered parameters than there are categories for the unordered categorical parameters.
[INFO 11-07 11:14:34] ax.modelbridge.dispatch_utils: Calculating the number of remaining initialization trials based on num_initialization_trials=None max_initialization_trials=None num_tunab

Epoch: 0, Training Loss | Acc: 1.144 0.580, Validation Loss | ACC : 0.630 0.789
Epoch: 1, Training Loss | Acc: 0.600 0.818, Validation Loss | ACC : 0.638 0.790
Epoch: 2, Training Loss | Acc: 0.567 0.823, Validation Loss | ACC : 0.603 0.790
Epoch: 3, Training Loss | Acc: 0.553 0.825, Validation Loss | ACC : 0.617 0.789
Epoch: 4, Training Loss | Acc: 0.549 0.824, Validation Loss | ACC : 0.578 0.791
Epoch: 5, Training Loss | Acc: 0.545 0.824, Validation Loss | ACC : 0.626 0.789
Epoch: 6, Training Loss | Acc: 0.539 0.825, Validation Loss | ACC : 0.591 0.790
Epoch: 7, Training Loss | Acc: 0.539 0.824, Validation Loss | ACC : 0.594 0.791
Epoch: 8, Training Loss | Acc: 0.536 0.824, Validation Loss | ACC : 0.577 0.795
Epoch: 9, Training Loss | Acc: 0.536 0.824, Validation Loss | ACC : 0.635 0.789
Epoch: 10, Training Loss | Acc: 0.536 0.825, Validation Loss | ACC : 0.600 0.794
Epoch: 11, Training Loss | Acc: 0.534 0.825, Validation Loss | ACC : 0.610 0.793
Epoch: 12, Training Loss | Acc: 0.533 

In [19]:
best_objectives = np.array(
    [[trial.objective_mean for trial in experiment.trials.values()]]
)

best_objective_plot = optimization_trace_single_method(
    y=np.minimum.accumulate(best_objectives, axis=1),
    title="Model performance vs. # of iterations",
    ylabel="CrossEntropy loss",
)
# render(best_objective_plot)

# render(plot_contour(model=model, param_x='dropout', param_y='lr', metric_name='MAE loss'))

In [20]:
data = experiment.fetch_data()
df = data.df
best_arm_name = df.arm_name[df["mean"] == df["mean"].min()].values[0]
best_arm = experiment.arms_by_name[best_arm_name]
best_arm

Arm(name='0_0', parameters={'neurons': 60, 'layers': 2, 'lr': 0.0017885402417238043, 'batchsize': 32, 'dropout': 0.34408897161483765})

## 4. Re-Train model with tuned hyperparameters

In [21]:
# Creating the LSTM class
class LSTM_direct_model(nn.Module):
    #  Determine what layers and their order in CNN object
    def __init__(self, hidden_dim, num_layers, droppout_prob):
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        super(LSTM_direct_model, self).__init__()
        self.lstm = nn.LSTM(
            input_size=num_features,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=droppout_prob,
        )
        self.fc = nn.Linear(hidden_dim, len(list_activities))

    # Progresses data across layers
    def forward(self, x):
        batch_size = x.size(0)
        init_states, init_cells = self.init_hidden(batch_size)
        init_states = init_states.to(x.device)
        init_cells = init_cells.to(x.device)
        lstm_output, (last_Hidden_State, last_Cell_State) = self.lstm(
            x, (init_states, init_cells)
        )
        out = self.fc(last_Hidden_State[-1])
        return out

    def init_hidden(self, batch_size):
        init_states = []
        init_cells = []
        for i in range(self.num_layers):
            init_states.append(torch.zeros(batch_size, self.hidden_dim))
            init_cells.append(torch.zeros(batch_size, self.hidden_dim))
        return torch.stack(init_states, dim=0), torch.stack(
            init_cells, dim=0
        )  # (num_layers, B, H)

In [22]:
batch_size = 64  # best_arm.parameters['batchsize']

train_loader = DataLoader(
    EventLogData(torch.tensor(X_train), torch.tensor(Y_train)),
    batch_size=batch_size,
    shuffle=True,
)

In [23]:
model_name = "_Tax_LSTM_direct"
save_folder = project_dir + "/5_Output_files/Next_Activity/" + data_name + model_name

if not os.path.exists(save_folder):
    os.mkdir(save_folder)

In [24]:
hidden_dim = 40  #  best_arm.parameters['neurons']
num_layers = 1  # best_arm.parameters['layers']
droppout_prob = 0.2  #  best_arm.parameters['dropout']
lr_value = 0.001  # best_arm.parameters['lr']
min_delta = 0
num_epochs = 100
early_stop_patience = 20
num_runs = 1
# Define loss and optimizer
for run in range(num_runs):
    print("Run: {}".format(run + 1))
    model = LSTM_direct_model(hidden_dim, num_layers, droppout_prob)
    model.to(dtype=dtype, device=device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr_value)
    epochs_plt = []
    cross_entropy_plt = []
    valid_loss_plt = []
    not_improved_count = 0
    # Train Network
    start_time = time.time()
    metric_tracker = ClassificationMetrics(len(list_activities))
    for epoch in range(num_epochs):
        model.train()
        training_loss = 0
        num_train = 0
        metric_tracker.clear()
        for inputs, labels in train_loader:
            # move data to proper dtype and device
            inputs = inputs.to(dtype=dtype, device=device)
            labels = torch.tensor([torch.max(yi, 0)[1] for yi in labels])
            labels = labels.to(device=device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            output = model(inputs)
            loss = criterion(output, labels)
            # back prop
            loss.backward()
            # optimize
            optimizer.step()
            training_loss += loss.item()
            num_train += 1
            _, preds = torch.max(output, 1)
            preds = preds.to(device)
            metric_tracker.add(preds, labels)
        train_acc = metric_tracker.acc()
        metric_tracker.clear()

        with torch.no_grad():
            model.eval()
            num_valid = 0
            validation_loss = 0
            for i, (inputs, targets) in enumerate(valid_loader):
                inputs, targets = inputs.to(device), torch.tensor(
                    [torch.max(yi, 0)[1] for yi in targets]
                ).to(device)
                yhat_valid = model(inputs)
                loss_valid = criterion(yhat_valid, targets)
                validation_loss += loss_valid.item()
                num_valid += 1
                _, preds = torch.max(yhat_valid, 1)
                preds = preds.to(device)
                metric_tracker.add(preds, targets)
        val_acc = metric_tracker.acc()

        avg_training_loss = training_loss / num_train
        avg_validation_loss = validation_loss / num_valid
        print(
            "Epoch: {}, Training Loss | Acc: {:.3f} {:.3f}, Validation Loss | ACC : {:.3f} {:.3f}".format(
                epoch, avg_training_loss, train_acc, avg_validation_loss, val_acc
            )
        )
        epochs_plt.append(epoch + 1)
        cross_entropy_plt.append(avg_training_loss)
        valid_loss_plt.append(avg_validation_loss)
        if epoch == 0:
            best_loss = avg_validation_loss
            torch.save(
                model.state_dict(),
                "{}/best_model_run_{}.pt".format(save_folder, run + 1),
            )
            best_model = copy.deepcopy(model)
        else:
            if best_loss - avg_validation_loss >= min_delta:
                torch.save(
                    model.state_dict(),
                    "{}/best_model_run_{}.pt".format(save_folder, run + 1),
                )
                best_model = copy.deepcopy(model)
                best_loss = avg_validation_loss
                not_improved_count = 0
            else:
                not_improved_count += 1
        # Early stopping
        if not_improved_count == early_stop_patience:
            print(
                "Validation performance didn't improve for {} epochs. "
                "Training stops.".format(early_stop_patience)
            )
            break
    training_time = time.time() - start_time
    print("Training time:", training_time)
    filepath = "{}/Loss_".format(save_folder) + data_name + "_run{}.txt".format(run)
    with open(filepath, "w") as file:
        for item in zip(epochs_plt, cross_entropy_plt, valid_loss_plt):
            file.write("{}\n".format(item))
        file.write("Running time: {}\n".format(training_time))

Run: 1
Epoch: 0, Training Loss | Acc: 1.550 0.481, Validation Loss | ACC : 1.088 0.620
Epoch: 1, Training Loss | Acc: 1.128 0.627, Validation Loss | ACC : 0.834 0.815
Epoch: 2, Training Loss | Acc: 0.879 0.706, Validation Loss | ACC : 0.702 0.831
Epoch: 3, Training Loss | Acc: 0.716 0.781, Validation Loss | ACC : 0.692 0.791
Epoch: 4, Training Loss | Acc: 0.640 0.818, Validation Loss | ACC : 0.658 0.791
Epoch: 5, Training Loss | Acc: 0.602 0.824, Validation Loss | ACC : 0.624 0.791
Epoch: 6, Training Loss | Acc: 0.582 0.825, Validation Loss | ACC : 0.613 0.791
Epoch: 7, Training Loss | Acc: 0.566 0.826, Validation Loss | ACC : 0.611 0.791
Epoch: 8, Training Loss | Acc: 0.558 0.825, Validation Loss | ACC : 0.626 0.791
Epoch: 9, Training Loss | Acc: 0.551 0.824, Validation Loss | ACC : 0.629 0.791
Epoch: 10, Training Loss | Acc: 0.554 0.825, Validation Loss | ACC : 0.611 0.791
Epoch: 11, Training Loss | Acc: 0.552 0.825, Validation Loss | ACC : 0.612 0.791
Epoch: 12, Training Loss | Acc:

## 5. Evaluation

In [25]:
lines, lines_t, lines_t2, lines_t3, lines_t4 = Extract_trace_and_temporal_features(
    tab_test
)
prefixes, outputs = Extract_prefix(lines, lines_t, lines_t2, lines_t3, lines_t4)

In [26]:
def evaluate_model(model):
    err_dict = {}
    with torch.no_grad():
        model.eval()

        metric_tracker = ClassificationMetrics(num_classes=len(list_activities))
        for i, (inputs, targets) in enumerate(test_loader):
            metric_tracker.clear()
            prefix_len = len(prefixes[0][i])

            # inputs = [[sub_item.to(dtype=torch.float32, device=device) for sub_item in item] for item in inputs]
            # targets = torch.tensor(targets).to(device=device)
            inputs = inputs.to(device)
            targets = torch.tensor([torch.max(yi, 0)[1] for yi in targets]).to(device)
            outputs = model(inputs)

            # loss_mape = torch.abs((targets - yhat)/targets)*100
            # criterion = nn.CrossEntropyLoss()
            # loss_mae = criterion(yhat,targets).item()

            metric_tracker.add(torch.max(outputs, 1)[1], targets)

            if prefix_len not in err_dict.keys():
                err_dict[prefix_len] = [metric_tracker.acc()]
            else:
                err_dict[prefix_len].append(metric_tracker.acc())
    return err_dict

In [27]:
err_total_dict = {}
print(save_folder)
for run in range(num_runs):
    print("Run: {}".format(run + 1))
    trained_model = LSTM_direct_model(hidden_dim, num_layers, droppout_prob)
    trained_model = trained_model.to(device)
    trained_model.load_state_dict(
        torch.load(
            "{}/best_model_run_{}.pt".format(save_folder, run + 1),
            map_location=torch.device(device),
        )
    )
    err_dict = evaluate_model(trained_model)
    print(err_dict)
    for key in err_dict.keys():
        err = torch.mean(torch.tensor(err_dict[key]), axis=0)
        if key in err_total_dict.keys():
            err_total_dict[key].append(torch.tensor(err))
        else:
            err_total_dict[key] = [torch.tensor(err)]

/home/sebdis/ProcessMining/Next_Activity_GNN/DuongNA//5_Output_files/Next_Activity/Helpdesk_Tax_LSTM_direct
Run: 1
{2: [tensor(0.), tensor(0.), tensor(1.), tensor(1.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(1.), tensor(0.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(0.), tensor(0.), tensor(0.), tensor(1.), tensor(0.), tensor(0.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(0.), tensor(1.), tensor(1.), tensor(1.), tensor(0.), tensor(0.), tensor(1.), tensor(1.), tensor(0.), tensor(0.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(0.), tensor(1.), tensor(1.), tensor(0.), tensor(0.), tensor(1.), tensor(0.), tensor(0.), tensor(0.), tensor(1.), tensor(0.), tensor(0.), tensor(1.), tensor(1.), tensor(1.), tens

In [28]:
num_samples_dict = {}
for i, (inputs, targets) in enumerate(test_loader):
    key = len(prefixes[0][i])
    if key in num_samples_dict.keys():
        num_samples_dict[key] += 1
    else:
        num_samples_dict[key] = 1

In [29]:
list_prefix_len = []
list_num_samples = []
list_accuracy = []

for key, value in err_total_dict.items():
    list_prefix_len.append(key)
    list_num_samples.append(num_samples_dict[key])
    list_accuracy.append(value[0].item())

tab_result = pd.DataFrame(
    {
        "Prefix length": list_prefix_len,
        "Num samples": list_num_samples,
        "Accuracy(%)": list_accuracy,
    }
)
tab_result

,Prefix length,Num samples,Accuracy(%)
0,2,1518,0.571146
1,3,1450,0.646207
2,4,684,0.821637
3,5,173,0.699422
4,6,68,0.779412
5,7,20,0.900000
6,8,4,0.750000
7,9,1,1.000000
8,10,1,0.000000
9,11,1,0.000000


In [30]:
tab = tab_result[tab_result["Num samples"] >= 20]
general_acc = round(tab["Accuracy(%)"] * tab["Num samples"])
# print(general_acc)
print(sum(general_acc) / sum(tab["Num samples"]))

0.6537183746486072


In [32]:
tab_result.to_csv(
    project_dir
    + "4_Outputs/Evaluation/"
    + data_name
    + "_Next_Activity_Tax_LSTM_eval.csv",
    index=False,
)